# Project Mobile robotics
## filtering

what the fuck do we want?
Well a filter means having an estimated position at all time. And if we move we estimate where it is now, and merge this belief with what we see on the camera

And then we can always estimate where we are even if the camera is not on its best behavior


To have the position of robot we need x and y coordinates and orientation 
- sensing and performing a measurement update 
- moving and performing a motion update

In [2]:
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=d392bffcd6af1e3c5b48b013843495b364de7381ac638cc13d77c4232a10a846
  Stored in directory: /Users/Julie/Library/Caches/pip/wheels/53/e6/de/a09ea01e923aaf88b9f8c7c44329e857b2c1a31901167e55e6
Successfully built filterpy


In [3]:
import numpy as np
from filterpy.kalman import KalmanFilter

T_s = 1 # period en secondes selon laquelle on prend les mesures

# measurements i need from paul !! 
inital_x = 0
initial_y = 0 

measurement_noise_var = 1
model_noise_var = 1

# Kalman filter using module (well this was linear so fuck)
- x - state: x, y, vx, vy (position and speed)
- z - measurements (z): x, y, o (position and orientation)
- P - covariance matrix: is updated to show the uncertainties of each states (describe how much each pair of state variables is correlated and how much uncertainty there is in the estimation of each individual state variable)
- F - state transition: dont know how to do it
- H- measurement: Identity as we measure x,y,o
- R- noise, uncertainty of measurements
- Q- noise, uncertainty of my model

In [ ]:
def create_kalman(initial_x, initial_y, initial_o, measurement_noise_var, model_noise_var):
    # create the filter
    dim_x = 3
    dim_z = 3
    kf = KalmanFilter(dim_x=dim_x, dim_z=dim_z)  # 4 states (x, y, o), 3 measurements (x, y, o)

    kf.x = np.array([inital_x, initial_y, initial_o]) # assuming we start when its stopped
    kf.P = np.eye(dim_x) # use identity for now

    # state transition model:
    kf.F = None # its a mess

    # measurements: 
    kf.H = np.eye(dim_z)

    # noise, can use variance of measurements ? NEED TO WORK MORE ON THIS
    kf.R = np.eye(dim_z) * measurement_noise_var
    kf.Q = np.eye(dim_x) * model_noise_var
    return kf

In [ ]:
def updating_filter(measurement):
    # Prediction step
    kf.predict()

    # Correction step
    kf.update(measurement)

    # Updated position estimate
    estimated_position = kf.x

    return estimated_position

# extended kalman filter (non linear!)

In [8]:
!pip install autograd

  Obtaining dependency information for autograd from https://files.pythonhosted.org/packages/81/70/d5c7c2a458b8be96495c8b1634c2155beab58cbe864b7a9a5c06c2e52520/autograd-1.6.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 448.8 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 954.1 kB/s eta 0:00:000:00:01
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492024 sha256=9a905e1c27e6a05fb5fff04a28ce188255ffc1aba51ac3f98af7da44cfe16729
  Stored in directory: /Users/Julie/Library/Caches/pip/wheels/bf/5d/6a/2e53874f7ec4e2bede522385439531fafec8fafe005b5c3d1b
Successfully built future


In [9]:
from filterpy.kalman import ExtendedKalmanFilter
from autograd import jacobian

import numpy as np

def create_EKF(initial_x, inital_y, initial_o, model_noise_var, measu_noise_var):
    """ Initializes the Extended Kalman Filter.
    Args: initial position, noise variance for measurement and model
    Returns: the Extended Kalman Filter
    """
    # Define the state transition function for your system
    def state_transition_function(x, u, dt):
        """ x[0]: x position, x[1]: y position, x[2]: orientation
            u[0]: linear velocity, u[1]: angular velocity
        """
        print("u")
        print(u[0])
        new_x = x[0] + u[0]
        x[0] += u[0] * np.cos(x[2]) * dt
        x[1] += u[0] * np.sin(x[2]) * dt
        x[2] += u[1] * dt
    
        print("inside state tranisition")
        print(x)
        return x

    # Define the measurement function for your system
    def measurement_function(measurements):
        """Computes the H matrix
        Args: measurements (array) containing x, y and o
        returns: H matrix, with measurements
        """
        H = np.array([
            [measurements[0], 0, 0],
            [0, measurements[1], 0],
            [0, 0, measurements[2]]
        ])
        return H

    dim_x = 3 # cause we have x,y,orientation as state
    dim_z = 3 # cause we have x, y, orientation as measurements
    ekf = ExtendedKalmanFilter(dim_x=dim_x, dim_z=dim_z, dim_u=2)

    # Set the initial state and covariance
    ekf.x = np.array([initial_x, inital_y, initial_o])
    ekf.P = np.eye(dim_x) # initial for now

    # Set the state transition function and measurement function
    ekf.F = state_transition_function
    ekf.H = measurement_function

    # Set the process noise covariance Q and measurement noise covariance R
    ekf.Q = np.eye(dim_x) * model_noise_var
    ekf.R = np.eye(dim_z) * measu_noise_var

    return ekf

In [17]:
def update_ekf(ekf, measurement, speed):
    """ updates the EKF and computes the estimated position at this state
    Args: 
        ekf: the EKF object that has been initialized
        measurement: numpy array with x, y, o
        speed: numpy array with speed and (linear and angular)

    """
    print("pre update")
    print(ekf.x)
    #potentialy have to do some work on speed, but i don't know what it looks like yet
    # ekf.u = speed
    ekf.predict(u=speed)
    def jac(x):
        return np.eye(len(x))
    
    # Expected measurement at the predicted state
    ekf.update(measurement, jac, Hx=ekf.H)

    updated_state = ekf.x
    print("post update")
    print(ekf.x)
    return updated_state

## trying to run ? 

In [18]:
ekf= create_EKF(0,0,0,1,1)

for i in range(2):
    print("i")
    print(i)
    update_ekf(ekf, np.array([0,0,0]), np.array([0,0]))
    print(ekf.x)


i
0
pre update
[0 0 0]


TypeError: unsupported operand type(s) for *: 'function' and 'int'

In [30]:
initial_x = 0
initial_y = 0
initial_o = 180

model_noise_var = 1.2
measu_noise_var = 1
ekf = create_EKF(initial_x, initial_y, initial_o, model_noise_var, measu_noise_var)

measurements = np.array([
    [0,1,180],
    [0,1,90]
])

us = np.array([
    [1, 0],
    [0, 90]
])

print("estimated position after 1 \n")
print(update_ekf(ekf, measurements[0], np.array([1,0])))
print("estimated position after 2 \n")
print(update_ekf(ekf, measurements[1], np.array([0, 90])))

[  0   0 180]
estimated position after 1 

[[  0.       0.6875 303.75  ]
 [  0.       0.6875 303.75  ]
 [  0.       0.6875 180.    ]]
estimated position after 2 

[[array([   0.        ,   -0.44940476, -198.55519481])
  array([1.34117965, 1.34117965, 1.34117965])
  array([362.58116883, 362.58116883, 362.58116883])]
 [array([0., 0., 0.])
  array([   1.34117965,    0.89177489, -197.21401515])
  array([362.58116883, 362.58116883, 362.58116883])]
 [array([0., 0., 0.]) array([1.34117965, 1.34117965, 1.34117965])
  array([238.83116883, 238.38176407, 121.16883117])]]


/var/folders/_j/j0g7xy7j45sb22pt5plmfvsh0000gn/T/ipykernel_23921/4273059487.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  H = np.array([


In [48]:
initial_pos = np.array([0,0,0])
initial_speed = np.array([0,0])
measurements = np.array([
    [1,0,0],
    [1,0,90]
])

us = np.array([
    [1, 0],
    [0, 90]
])
ekf = EKF(1, initial_pos, initial_speed)
ekf.filter(measurements[0], us[0])
ekf.filter(measurements[1], us[1])


State Estimate Before EKF=[1.01  0.01  0.003]
Observation=[1 0 0]
State Estimate After EKF=[ 0.968 -0.032 -0.02 ]
State Estimate Before EKF=[ 0.978 -0.022 89.983]
Observation=[ 1  0 90]
State Estimate After EKF=[ 0.949 -0.051 89.969]
